# Mobile Robotics Project - Final Report

Group Members:
- ESCOYEZ, Antoine Jacques Richard, 335564
- POUSSIN, Jean-Baptiste Marie Alexandre, 303127
- KOCKISCH, Matthias Hugues Jörg, 303000
- STUBER, Lukas, 289304

Professor:

Prof. MONDADA, Francesco


EPFL, 11.12.2022

Course MICRO-452 Basics of Mobile Robotics

# Project Description
The goal of the project in the Basics of Mobile Robotics class was to control the two sheel Thymio robot. The environment of the robot is a map designed by the group members. On the map there need to be obstacles into which the robot cannot/must not drive. According to the chosen map and the obstacles present on the map a global path from the start to the goal needs to be calculated using computer vision. The start and initial position (*i.e.* the location of Thymio,) can be chosen by the user. On its path to the goal the robot must be be able to detect and avoid unforeseen obstacles that are added by the user. For better tracking of the path some kind of filtering must be used that improves the state estimation of the robot. The camera image serves as measurement for the filter.

# Detailed Description of our Choices
Our group chose the map to be a parking space with two rows of parking lots. The yellow lines marking the edges of the parking lots are the fixed obstacles used in the global path planning. The robot is neighter allowed to cross the yellow lines nor leave the map. The camera is fixed above the map and oversees the entire situation (robot, lines, free spaces, occupied spaces). To facilitate the detection of the position and orientation of the robot in the images we sticked two coloured stickers on the top side of the robot. The left sticker is green, the right one is blue. The computer vision algorithm uses these two points to determine the center of the wheels axis and the orientation of the robot. The goal is a free parking lot indicated by a red paper square. The unknown obstacles (unknown to/ignored by the computer vision) are <font color='red'>????</font>
. The estimated state correction is done with the help of a Kalman filter whose measurement is the location of the Thymio in the camera images.


<img src="Map.jpg" width="300">

# Computer Vision
We use a *AUKEY 1080P - 30 pfs* camera as the optical sensor for the computer vision. It is fixed on a stand (a lamp) and sees the entire map from above. Its position does not change during the operation of the robot.

To enable a clear and stable detection of the objects on the map a series of adjustments must be done before the object detection. The image processing is done in HSV representation of the image. The procedure is automatically started when the main function is executed. It includes:
- Adjust the luminosity <font color='red'>????</font>
- Detection of the outer borders of the map
- Adjust the red upper and lower thresholds <font color='red'>????</font>
- Adjust the green upper and lower thresholds <font color='red'>????</font>
- Adjust the blue upper and lower thresholds <font color='red'>????</font>
- Adjust the yellow upper and lower thresholds <font color='red'>????</font>

<font color='red'>Describe the steps of the computer vision to find the colorized map.</font>

The computer vision algorithm abstracts the picture of the map leaving only the relevant objects (map surface, robot (green and blue spot), obstacle lines, goal) in full colors (rgb values eighter 255 or 0).

# Global Path Planning
The global path planning is done using the A* algorithm, as seen in Exercise 5 of the course. It uses the simplified map generated by the computer vision part as input. The typical size of this image is <font color='red'>????</font>. Obviously, this is far too large for the A* algorithm to finish in an acceptable time. To reduce the size of the image a convolutional filter is applied to the pixels. Its size is can be chosen by the user and is represented by the variable "kernel". The filter devides the image from the vision part into patches of size 2*kernel+1 by 2*kernel+1. The output pixel is 0 if:
- all the pixels in the patch are black (free space)
- at least one of the pixels is green or blue (robot position)
- at least one of the pixels is red (goal position)

The output is 1 if:
- at least one of the pixels is yellow (obstacle line)

The result of this filtering is a map of typical size 120 by 60 pixels (depending on the size of the input map) where the free, start and goal space is 0 and the obstacle space is 1.

This reduced map and the start and goal positions are fed into the A* algorithm that calculates the optimal global path from the start to the goal. The calculated path is a series of coordinates in the reduced map format. To serve as directions for the Thymio the coordinates need to be resized to the original map (multiplied by the reduction factor 2*kernel+1).

# Motion Control
Input: path (series of coordinates)
Output: speed of the wheels

# Kalman Filter
The Kalmna Filter is quite standard. It is a class with the following attributes:
- the current state estimation, a vector with (x position, y position, orientation theta)
- the current state estimation covariance matrix
- the process uncertainty matrix (constant, describes the noise of the state propagation)
- the measurement uncertainty matrix (constant, describes the noise of the measurement by the camera)
- the observation matrix (constant, maps the state to the observation of the state), since all of the states can be measured, this is an identity matrix
- the time stamp of the last time the filter was called (used for calculating the elapsed time)

and the following methods:
- state_prop <br>
It takes as input the speeds of the two wheels during the last timestep. From that it estimates the new position and state covariance matrix and updates the corresponding attributes. Because the state propagation includes sinus and cosinus calculations, it is not a linear system. To facilitate the calculations a first order approximation is used that linearizes the system. The error of the approximation is limited because the state_prop function is called every 25ms <font color='red'>????</font>. So, the involved angle is very small and so is the error of the linearization. 
- state_correct <br>
It takes as input the measurement of the position and angle of the robot from the camera. It calculates the Kalman gain and corrects the state and state covariance. It takes quite some time to take an image with the camera and extract the position of the robot, which means that the correct function can only be called every 1.5s <font color='red'>????</font>.

# Local obstacles avoidance
Input: prox measurements
Output: speed of the wheels

# Main Function
bla bla, maybe talk about the timers

# Conclusion
conclude conclude

In [ ]:
# main function

from __future__ import print_function
import cv2 as cv
from parking_segmentation import *
from color_segmentation import *
from color_centroids import *
from discretize_map import *
from control import ThymioControl
from Kalman import Kalman
from constants import *
from RepeatedTimer import RepeatedTimer

# IMAGE PROCESSING
id_camera = 1
##[Parking segmentation]
corners, destination_corners = set_parking_limits(id_camera)
##[Color segmentation]
segmentation, refined_color_dict_HSV, kernels, openings = get_color_mask(id_camera, corners, destination_corners, real_size=(NOMINAL_AREA_LENGTH, NOMINAL_AREA_WIDTH))
cv.namedWindow("Segmentation Result")
cv.imshow("Segmentation Result", segmentation)
key = cv.waitKey(0)
cv.destroyWindow("Segmentation Result")
##[Thymio and objective localization]
centroids = {'goal': (0, 0), 'thymio': (0, 0), 'green': (0, 0), 'blue': (0, 0)}
theta_thymio = 0
localization = None

kalman = Kalman()
thymio = ThymioControl()

def compute_centroids():
    global centroids, theta_thymio, localization
    centroids, theta_thymio, localization = get_centroids(id_camera, corners, destination_corners, refined_color_dict_HSV, kernels, openings, prev_centroids=centroids, real_size=(NOMINAL_AREA_LENGTH, NOMINAL_AREA_WIDTH), real_time=False)
    #print("centroids at", centroids['thymio'][0], centroids['thymio'][1])
    #print("thymio angle at ", theta_thymio)
    kalman.state_correct(np.array([centroids['thymio'][0], centroids['thymio'][1], theta_thymio]))
    thymio.position = (kalman.x[0, 0], kalman.x[1, 0])
    thymio.angle = kalman.x[2, 0]

def odometry():
    kalman.state_prop(thymio.speed_target)
    thymio.position = (kalman.x[0, 0], kalman.x[1, 0])
    thymio.angle = kalman.x[2, 0]
    #print(thymio.position[0], thymio.position[1], thymio.angle*180/math.pi)

# def plot_localization():
#     global localization
#     cv.namedWindow("Localization Result")
#     cv.imshow("Localization Result", localization)
#     key = cv.waitKey(30)
#     cv.destroyWindow("Localization Result")

# initialise position and path
compute_centroids()
kalman.set_state((centroids['thymio'][0], centroids['thymio'][1], theta_thymio))
thymio.position = (centroids['thymio'][0], centroids['thymio'][1])
thymio.angle = theta_thymio
path = discretize_map(segmentation, centroids)
thymio.set_path(path)

# start updating position and follow path
image_timer = RepeatedTimer(1.5, compute_centroids)
odometry_timer = RepeatedTimer(ODOMETRY_INTERVAL, odometry)
image_timer.start()
odometry_timer.start()
thymio.follow_path()